In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date

In [2]:
freq = ['daily', 'weekly', 'monthly']
files = [f'../Data/gspc_{f}_clean.csv' for f in freq]
dfs = [pd.read_csv(file) for file in files]

for df in dfs:
    df['date'] = pd.to_datetime(df['date']).dt.normalize()

daily = dfs[0]
weekly = dfs[1]
monthly = dfs[2]

In [3]:
t = pd.read_csv(f'../Data/tweet_df_cleaned_w_sentiment.csv')
t.rename(columns={
        'text': 'tweet', 
        'avg_scores': 'avg', 
        'sum_scores': 'sum', 
        'date': 'timestamp',
        'sentiment_category': 'sentiment',
        'word_count': 'words'
        }, inplace=True)
#tweets.columns
columns = [
        'timestamp', 
        'tweet', 
        'avg', 
        'sum', 
        'isRetweet', 
        'isDeleted', 
        'favorites', 
        'retweets',
        'sentiment',
        'words'
        ]

t['isRetweet'] = t['isRetweet'].apply(lambda x: 1 if x == 't' else 0)
t['isDeleted'] = t['isDeleted'].apply(lambda x: 1 if x == 't' else 0)
t['date'] = pd.to_datetime(t['timestamp']).dt.normalize()
t['month'] = t['date'] + pd.offsets.MonthBegin(-1)
t['year'] = pd.DatetimeIndex(t['timestamp']).year

avg_sent = t[(t['isRetweet'] == 'f') & (t['isDeleted'] == 'f') & (t['year'] >= 2016)]['avg'].mean()
t['delta_avg'] = t['avg'].apply(lambda x: x - avg_sent)

sentiments = ['positive', 'negative', 'neutral']
for s in sentiments:
        t[s] = np.where(t['sentiment'] == s, 1, 0)

m = t[t['year'] >= 2016]\
        .groupby('month')\
        .agg({
                'tweet': 'count',
                'isRetweet': 'sum',
                'isDeleted': 'sum', 
                'avg': 'mean', 
                'sum': 'sum',
                'delta_avg': 'mean', 
                'retweets': 'sum', 
                'favorites': 'sum', 
                'words': 'sum',
                'positive': 'sum',
                'negative': 'sum',
                'neutral': 'sum'
        })\
        .reset_index()

d = t[t['year'] >= 2016]\
        .groupby('date')\
        .agg({
                'tweet': 'count',
                'isRetweet': 'sum',
                'isDeleted': 'sum', 
                'avg': 'mean', 
                'sum': 'sum',
                'delta_avg': 'mean', 
                'retweets': 'sum', 
                'favorites': 'sum', 
                'words': 'sum',
                'positive': 'sum',
                'negative': 'sum',
                'neutral': 'sum'
        })\
        .reset_index()


for s in sentiments:
    d[f'%_{s}'] = d[s] / d['tweet']
    m[f'%_{s}'] = m[s] / m['tweet']


mo = pd.merge(m, monthly, how='inner', left_on='month', right_on='date')
mo = mo[['month', 'tweet', 'isRetweet', 'isDeleted', 'avg', 'sum', 'delta_avg', 'retweets', 'favorites', 'words', 'positive', 'negative', 'neutral', '%change', 'up/down']]\
        .reset_index()

da = pd.merge(d, daily, how='inner', on='date')
da = da[['date', 'tweet', 'isRetweet', 'isDeleted', 'avg', 'sum', 'delta_avg', 'retweets', 'favorites', 'words', 'positive', 'negative', 'neutral', '%change', 'up/down']]\
        .reset_index()

In [4]:
mo.rename(columns={
    'tweet': 'tweets',
    'avg': 'avg_sentiment_score',
    'sum': 'sum_sentiment_score',
    'words': 'word_count',
    '%change': 's&p_%change',
    'up/down': 's&p_up/down'
}, inplace=True)

da.rename(columns={
    'tweet': 'tweets',
    'avg': 'avg_sentiment_score',
    'sum': 'sum_sentiment_score',
    'words': 'word_count',
    '%change': 's&p_%change',
    'up/down': 's&p_up/down'
}, inplace=True)

In [11]:
# mo.to_csv('../Data/monthly_summary.csv', index=False)
# da.to_csv('../Data/daily_summary.csv', index=False)

In [5]:
mo.head()

,index,month,tweets,isRetweet,isDeleted,avg_sentiment_score,sum_sentiment_score,delta_avg,retweets,favorites,word_count,positive,negative,neutral,s&p_%change,s&p_up/down
0,0,2016-12-01,140,11,1,5.789808,8038.02,NaN,1956286,7467975,8174,102,9,28,0.017571,up
1,1,2017-01-01,212,5,2,5.604845,12097.93,NaN,3626958,17156762,12673,118,24,70,0.012125,up
2,2,2017-02-01,153,1,7,5.551829,8518.98,NaN,2784536,13958598,9001,79,29,44,0.034149,up
3,3,2017-03-01,152,13,9,5.694078,8317.00,NaN,2112539,9328782,8605,97,12,42,-0.007315,down
4,4,2017-04-01,146,13,6,5.696006,8166.45,NaN,1631378,7374391,8465,91,13,38,0.009253,up


In [11]:
t = pd.read_csv(f'../Data/tweet_df_cleaned_w_sentiment.csv')
t['word_and_scores'].head()

0    [('Republicans', 4.22), ('Democrats', 5.5), ('...
1    [('I', 5.92), ('back', 5.18), ('Great', 7.88),...
2    [('RT', 4.88), ('READ', 6.52), ('Letter', 5.94...
3    [('The', 4.98), ('Mail', 6.36), ('In', 5.5), (...
4    [('RT', 4.88), ('Very', 6.12), ('friendly', 7....
Name: word_and_scores, dtype: object